In [1]:
# encoding:utf-8
from utils import BayesianSmoothing, load_pickle, dump_pickle, raw_data_path
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
path = '../data/'
# 训练集
train = pd.read_table(path+'round1_ijcai_18_train_20180301.txt',encoding='utf8',delim_whitespace=True)
train['isTrain'] = 1
    
# 测试集
test = pd.read_table(path+'round1_ijcai_18_test_a_20180301.txt',encoding='utf8',delim_whitespace=True)
test['isTrain'] = 0
        
# 连接
df = pd.concat([train, test])
print("========> Load Data Success!") 

========> Load Data Success!


In [2]:
train.columns

Index([              u'instance_id',                   u'item_id',
              u'item_category_list',        u'item_property_list',
                   u'item_brand_id',              u'item_city_id',
                u'item_price_level',          u'item_sales_level',
            u'item_collected_level',             u'item_pv_level',
                         u'user_id',            u'user_gender_id',
                  u'user_age_level',        u'user_occupation_id',
                 u'user_star_level',                u'context_id',
               u'context_timestamp',           u'context_page_id',
       u'predict_category_property',                   u'shop_id',
           u'shop_review_num_level', u'shop_review_positive_rate',
                 u'shop_star_level',        u'shop_score_service',
             u'shop_score_delivery',    u'shop_score_description',
                        u'is_trade',                   u'isTrain'],
      dtype='object')

In [3]:
train.item_brand_id.value_counts()

7838285046767229711    69746
7066302540842412840    13823
5051039799137810159    13045
448955875785543916     10116
739382823914881928      6542
6793286343174833903     5280
1902568530584691432     4605
4609398715869550725     4235
6597576831450166037     4216
8188723660812005813     3739
5818352635473521241     3730
3801992748557162347     3474
7024713306861377790     3453
7166307405309656478     3195
3765442984647241130     3178
1212158767298395095     3097
1842327023449366624     3033
5368019250815240955     2990
6553461951829665552     2764
3884267719846612071     2678
3697251094784296011     2645
7393038778026345793     2607
4940851772115583463     2492
2441922225904422164     2456
3321064285232298680     2388
665010924190078800      2375
8036402958319042519     2369
6874110308416919683     2347
4219436286513843074     2339
2707400123518269628     2288
                       ...  
3095754226004529341        1
5454033227514511696        1
4688766976726863845        1
23545905335358

### item history click rate

In [8]:
item_click_rate = train.groupby(train['item_id'])['is_trade'].agg('mean').reset_index(name="item_click_rate")
df = pd.merge(df, item_click_rate, 'left', on='item_id')
df['item_click_rate'].fillna(df['item_click_rate'].mean(), inplace=True)

### shop history click rate

In [9]:
shop_click_rate = train.groupby(train['shop_id'])['is_trade'].agg('mean').reset_index(name="shop_click_rate")
df = pd.merge(df, shop_click_rate, 'left', on='shop_id')
df['shop_click_rate'].fillna(df['shop_click_rate'].mean(), inplace=True)

### shop feature

In [ ]:
shop_columns = ['shop_id', 'shop_review_positive_rate', 'shop_score_delivery',
                'shop_score_description', 'shop_score_service','shop_star_level']

### user history click rate User Portrait

In [27]:
user_property = ['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']
user_property_click_rate = train.groupby(user_property)['is_trade'].agg('mean').reset_index(name="user_property_click_rate")
df = pd.merge(df, user_property_click_rate, 'left', on=user_property)

In [34]:
user_property_click_rate.nunique()

user_gender_id                4
user_age_level                9
user_occupation_id            5
user_star_level              12
user_property_click_rate    374
dtype: int64

### item history click rate according the property

In [ ]:
item_property = []

### user_id click

In [106]:
user_click_rate = train.groupby(train['user_id'])['is_trade'].agg('mean').reset_index(name="user_click_rate")
df = pd.merge(df, user_click_rate, 'left', on='shop_id')
df.user_click_rate.isna().value_counts()